In [10]:
import sys, os
os.environ['JAX_PLATFORMS'] = 'cpu'

import jax
from typing import Dict
import jax.numpy as jnp
import optax
import numpy as np
import math
from matplotlib import pyplot as plt
from jax_tqdm import scan_tqdm
from functools import partial
from jax.tree_util import tree_map
from dataclasses import dataclass
import eindex.array_api as EX
import einops

from jaxtorch import nn
from jaxtorch import PRNG, Context
import jaxtorch

from my_little_optimizer_client import SpaceItem, OptimClient

import optuna

In [11]:
SWEEP_NAME='21-wooden-sweep'
study = optuna.create_study(
    study_name=SWEEP_NAME,
    load_if_exists=True,
    storage="sqlite:///../vqgan-jax/optuna.db",
    direction="minimize",
    sampler=optuna.samplers.GPSampler(),
)

pars = list(study.trials[0].params.keys())
dists = study.trials[0].distributions
trials = [trial for trial in study.trials if trial.state==1]
all_y = jnp.array([trial.value for trial in trials])
max_y = jnp.max(all_y[jnp.isfinite(all_y)])
# import json
# with open('../vqgan-jax/trials.json', 'r') as fp:
#     trials = json.load(fp)
#     all_y = jnp.array([trial[1] for trial in trials])
#     max_y = jnp.max(all_y[jnp.isfinite(all_y)])

# @dataclass
# class Trial:
#     params: Dict[str, float]
#     value: float

# trials = [Trial(params, value) for params, value in trials]
# pars = list(trials[0].params.keys())

/tmp/ipykernel_3067929/2264734566.py:7: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
  sampler=optuna.samplers.GPSampler(),
[I 2025-02-26 15:00:09,146] Using an existing study with name '21-wooden-sweep' instead of creating a new one.


In [12]:
import time
client = OptimClient('http://localhost:8000')
space = [
    SpaceItem(name, dist.low, dist.high, log=dist.log)
    for name, dist in (study.best_trial.distributions.items())
    if isinstance(dist, optuna.distributions.FloatDistribution)
]
sweep = client.new_sweep('vqgan-demo', space, objective='min', project_name='test2')
print('id:', sweep.id)
for trial in trials:
    a = sweep.start(trial.params)
    time.sleep(1.0)
    a.report(trial.value if jnp.isfinite(trial.value) else float(max_y))


id: e180103d-a14c-4b5b-9b00-7a39728461a1
